In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from General_Functions import long_horizon_ret
from Main import settings, features, pf_set
from pandas.tseries.offsets import MonthEnd
from Prepare_Data import process_risk_free_rate, process_return_data, wealth_func, load_and_filter_market_returns_test, process_all_data, process_cluster_labels
from General_Functions import size_screen_fun, addition_deletion_fun
import numpy as np
from scipy.stats import rankdata
import re
import gc

Features list length: 154


In [2]:
#Indhenter chars og daily:
file_path_usa_test = "./data_test/usa_test.parquet"
daily_file_path = "./data_test/usa_dsf_test.parquet"
file_path_world_ret = "./data_test/world_ret_test.csv"
risk_free_path = "./data_test/risk_free_test.csv"
market_path = "./data_test/market_returns_test.csv"

# Kald funktionen
chars, daily = process_all_data(file_path_usa_test, daily_file_path, file_path_world_ret, risk_free_path, market_path)
print(chars)
print(daily)

All missing excludes 0.60% of the observations
Filen er indlæst og filtreret succesfuldt.
No size screen
Turnover wo addition/deletion rule: 0.19%
Turnover w  addition/deletion rule: 33.72%
          id        eom   sic size_grp            me  rvol_252d  dolvol_126d  \
1      10104 2010-02-28  7372     mega  123619.74809   0.343023     0.854651   
2      10104 2010-03-31  7372     mega  129040.82501   0.267442     0.854651   
3      10104 2010-04-30  7372     mega  130101.18774   0.284884     0.848837   
4      10104 2010-05-31  7372     mega  113436.81847   0.354651     0.848837   
5      10104 2010-06-30  7372     mega  107857.95540   0.261628     0.843023   
...      ...        ...   ...      ...           ...        ...          ...   
28707  92655 2023-06-30  6324     mega  447491.23412   0.284884     0.953488   
28708  92655 2023-07-31  6324     mega  471446.66929   0.319767     0.953488   
28709  92655 2023-08-31  6324     mega  441458.42446   0.348837     0.941860   
28710  926

In [3]:
file_path_cluster_labels = "Data/Cluster Labels.csv"
file_path_factor_details = "Data/Factor Details.xlsx"
cluster_labels = process_cluster_labels(file_path_cluster_labels, file_path_factor_details)
print(cluster_labels)

       characteristic       cluster  direction
0                 age  low_leverage       -1.0
1             aliq_at    investment       -1.0
2            aliq_mat  low_leverage       -1.0
3            ami_126d          size        1.0
4               at_be  low_leverage       -1.0
..                ...           ...        ...
149           z_score  low_leverage        1.0
150  zero_trades_126d      low_risk        1.0
151   zero_trades_21d      low_risk        1.0
152  zero_trades_252d      low_risk        1.0
153         rvol_252d      low_risk       -1.0

[154 rows x 3 columns]


In [4]:
# Prepare cluster characteristics
valid_chars = chars[chars["valid"] == True]
cluster_data_m = valid_chars[["id", "eom", "size_grp", "ff12"] + features].copy()

clusters = cluster_labels["cluster"].unique()
cluster_ranks = {}

for cl in clusters:
    chars_sub = cluster_labels[(cluster_labels["cluster"] == cl) & (cluster_labels["characteristic"].isin(features))]

    data_sub = cluster_data_m[chars_sub["characteristic"].values].copy()

    for c in chars_sub["characteristic"].values:
        dir_value = chars_sub.loc[chars_sub["characteristic"] == c, "direction"].values[0]
        if dir_value == -1:
            data_sub[c] = 1 - data_sub[c]

    cluster_ranks[cl] = data_sub.mean(axis=1)

# Konverter cluster ranks til en dataframe
cluster_ranks_df = pd.DataFrame(cluster_ranks)

# Kombiner med de oprindelige data
cluster_data_m = cluster_data_m[["id", "eom", "size_grp", "ff12"]].copy()
cluster_data_m["eom_ret"] = cluster_data_m["eom"] + pd.DateOffset(months=1) - pd.DateOffset(days=1)

# Merge cluster rankings
cluster_data_m = pd.concat([cluster_data_m, cluster_ranks_df], axis=1)

# Output result
print(cluster_data_m.head())
print(cluster_data_m)

       id        eom size_grp   ff12    eom_ret  low_leverage  investment  \
24  10104 2012-01-31     mega  BusEq 2012-02-28      0.763518    0.497303   
25  10104 2012-02-29     mega  BusEq 2012-03-28      0.775977    0.498864   
26  10104 2012-03-31     mega  BusEq 2012-04-29      0.772860    0.593682   
27  10104 2012-04-30     mega  BusEq 2012-05-29      0.723207    0.556345   
28  10104 2012-05-31     mega  BusEq 2012-06-29      0.742939    0.566149   

        size     value   quality  low_risk  debt_issuance  seasonality  \
24  0.279070  0.364911  0.700778  0.531729       0.764053     0.407858   
25  0.283430  0.374391  0.702219  0.447165       0.764182     0.357189   
26  0.287791  0.400704  0.713801  0.457752       0.757182     0.306774   
27  0.286337  0.401420  0.712786  0.508242       0.776053     0.367950   
28  0.286337  0.445100  0.720561  0.387598       0.776172     0.592911   

    accruals  profitability  profit_growth  short_term_reversal  momentum  
24  0.400278    